# Loan subgrade prediction modeling

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
import plotly.offline as pyo
import plotly.express as px
import plotly.graph_objects as go
import math

import scipy.stats.distributions as dist
from statsmodels.stats.proportion import proportions_ztest
from scipy import stats

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from generic_functions import value_counts_to_dataframe, pivot_dim2
from modeling_functions_multi_class import fit_model, kfold_cross_validation

# balancing dataset
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

from sklearn import model_selection
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, GridSearchCV, StratifiedKFold

from sklearn import metrics
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from scikitplot.metrics import plot_roc_curve
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import precision_recall_curve


from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.ensemble import GradientBoostingClassifier
#from xgboost import XGBRegressor
import xgboost as xgb

from sklearn.pipeline import Pipeline

from scipy.stats import reciprocal, uniform

from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

os.getcwd()

'/Users/kristinazekiene/Documents/LearningPython/TuringCollege/M3S3/V2'

In [2]:
path_file_xtrain = f'/Users/kristinazekiene/Documents/LearningPython/TuringCollege/M3S3/V2/output/subgrade/x_train.csv'
path_file_xtest = f'/Users/kristinazekiene/Documents/LearningPython/TuringCollege/M3S3/V2/output/subgrade/x_test.csv'
path_file_ytrain = f'/Users/kristinazekiene/Documents/LearningPython/TuringCollege/M3S3/V2/output/subgrade/y_train.csv'
path_file_ytest = f'/Users/kristinazekiene/Documents/LearningPython/TuringCollege/M3S3/V2/output/subgrade/y_test.csv'

# path_file_xtrain = f'/Users/kristinazekiene/Documents/LearningPython/TuringCollege/M3S3/V2/output/subgrade/sample/x_train.csv'
# path_file_xtest = f'/Users/kristinazekiene/Documents/LearningPython/TuringCollege/M3S3/V2/output/subgrade/sample/x_test.csv'
# path_file_ytrain = f'/Users/kristinazekiene/Documents/LearningPython/TuringCollege/M3S3/V2/output/subgrade/sample/y_train.csv'
# path_file_ytest = f'/Users/kristinazekiene/Documents/LearningPython/TuringCollege/M3S3/V2/output/subgrade/sample/y_test.csv'

x_train = pd.read_csv(path_file_xtrain)
x_test = pd.read_csv(path_file_xtest)
y_train = pd.read_csv(path_file_ytrain)
y_test = pd.read_csv(path_file_ytest)
x_train.head(3)

,year,grade,loan_amnt,installment,annual_inc,dti,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,delinq_2yrs,mths_since_last_delinq,term,emp_length,ohe__emp_title_agg_accountant,ohe__emp_title_agg_analyst,ohe__emp_title_agg_director,ohe__emp_title_agg_driver,ohe__emp_title_agg_engineer,ohe__emp_title_agg_legal,ohe__emp_title_agg_manager,ohe__emp_title_agg_nurse,ohe__emp_title_agg_other,ohe__emp_title_agg_owner,ohe__emp_title_agg_police,ohe__emp_title_agg_sales,ohe__emp_title_agg_supervisor,ohe__emp_title_agg_teacher,ohe__home_ownership_ANY,ohe__home_ownership_MORTGAGE,ohe__home_ownership_NONE,ohe__home_ownership_OTHER,ohe__home_ownership_OWN,ohe__home_ownership_RENT,ohe__verification_status_Not_Verified,ohe__verification_status_Source_Verified,ohe__verification_status_Verified,ohe__purpose_car,ohe__purpose_credit_card,ohe__purpose_debt_consolidation,ohe__purpose_educational,ohe__purpose_home_improvement,ohe__purpose_house,ohe__purpose_major_purchase,ohe__purpose_medical,ohe__purpose_moving,ohe__purpose_other,ohe__purpose_renewable_energy,ohe__purpose_small_business,ohe__purpose_vacation,ohe__purpose_wedding,ohe__application_type_Individual,ohe__application_type_Joint_App,ohe__addr_state_AK,ohe__addr_state_AL,ohe__addr_state_AR,ohe__addr_state_AZ,ohe__addr_state_CA,ohe__addr_state_CO,ohe__addr_state_CT,ohe__addr_state_DC,ohe__addr_state_DE,ohe__addr_state_FL,ohe__addr_state_GA,ohe__addr_state_HI,ohe__addr_state_IA,ohe__addr_state_ID,ohe__addr_state_IL,ohe__addr_state_IN,ohe__addr_state_KS,ohe__addr_state_KY,ohe__addr_state_LA,ohe__addr_state_MA,ohe__addr_state_MD,ohe__addr_state_ME,ohe__addr_state_MI,ohe__addr_state_MN,ohe__addr_state_MO,ohe__addr_state_MS,ohe__addr_state_MT,ohe__addr_state_NC,ohe__addr_state_ND,ohe__addr_state_NE,ohe__addr_state_NH,ohe__addr_state_NJ,ohe__addr_state_NM,ohe__addr_state_NV,ohe__addr_state_NY,ohe__addr_state_OH,ohe__addr_state_OK,ohe__addr_state_OR,ohe__addr_state_PA,ohe__addr_state_RI,ohe__addr_state_SC,ohe__addr_state_SD,ohe__addr_state_TN,ohe__addr_state_TX,ohe__addr_state_UT,ohe__addr_state_VA,ohe__addr_state_VT,ohe__addr_state_WA,ohe__addr_state_WI,ohe__addr_state_WV,ohe__addr_state_WY
0,10.0,2.0,9600.0,326.20,70000.0,18.219999,680.0,684.0,1.0,18.0,0.0,5661.0,26.900000,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5.0,5.0,20000.0,563.24,94547.0,26.320000,715.0,719.0,1.0,14.0,0.0,31097.0,88.099998,0.0,31.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8.0,1.0,2000.0,65.99,115000.0,11.450000,660.0,664.0,0.0,14.0,0.0,1339.0,19.400000,4.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Random forest

In [ ]:
%%time
rf = RandomForestClassifier(random_state=42)

res = fit_model('Random Forest',rf,x_train,y_train,x_test,y_test,'sub_grade')
# res_df = pd.concat([res_df,pd.DataFrame.from_dict(res,orient='index').T])
res_df = pd.DataFrame.from_dict(res,orient='index').T
kfold_cross_validation('Random Forest',rf,x_train,y_train)
res